In [1]:
import os
import sys
sys.path.append('../../')
from recs.backtest import get_backtest_answer
from recs.utils.path import *
import __init__
from recs.utils.common import predict_ranking, readjson2dict, recall_evaluate, id2cat
import recs
from tqdm import tqdm
import pandas as pd
tqdm.pandas()

In [5]:
u2idx = readjson2dict("crm_idx")
i2idx = readjson2dict("fund_info_idx")
tran_data = pd.read_pickle(DATA_PATH / "trans_buy.pkl")
tran_data

tran_data["id_number"] = tran_data.id_number.progress_apply(lambda x: id2cat(u2idx, x))
tran_data["fund_id"] = tran_data.fund_id.progress_apply(lambda x: id2cat(i2idx, x))

100%|██████████| 339551/339551 [00:36<00:00, 9188.69it/s]


In [6]:
trans_buy_train = tran_data
crm = pd.read_csv(DATA_PATH / "crm_for_clustering_2020.csv")
crm = crm[crm.yyyymm == 202012]

target_users = crm['身分證字號'].unique().tolist()
best_sells = trans_buy_train[
    (trans_buy_train.buy_date > 20181200 )
].fund_id.value_counts().rename_axis('fund_id').reset_index(name='count')
trans_buy_train

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (24) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,id_number,fund_id,buy_date,yyyymm
1,A2774324330,776,20170327,201702
3,A2774533490,FW1,20200820,202007
5,A2774639190,696,20190709,201906
6,A2774639190,C0A,20200303,202002
7,A2774639190,D56,20200707,202006
...,...,...,...,...
395707,Q2739381200,J21,20191114,201910
395708,Q2750784640,57R,20180509,201804
395709,Q2751114730,J0D,20180515,201804
395710,Q2751114730,L0C,20200131,201912


In [50]:
print(len(target_users))

154913


In [8]:
crm.columns

Index(['index', 'ETFAUM', '外幣定存AUM', '海外債AUM', 'IbrainAUM', '投資型保險AUM', '年收入',
       '台幣活存AUM', '年齡', '收入註記', '總投資理財AUM', '外幣庫存註記', 'Line理財標籤註記', '外幣活存AUM',
       '當年度債券AP', '當年度總AP合計', '身分證字號', '投資經驗註記', '台幣定存AUM', '儲蓄型保險AUM',
       '領股利註記', '存款AUM', '特金AUM', '當年度基金AP', 'CIFAOCODE', '持有信用卡註記',
       'SI/DCI_AUM', '信託戶註記', '保險AUM', '基金AUM', '當年度黃金存摺AP', '當年度人身保險總AP',
       '海外投資註記', '房貸壽險AUM', '當年度台外幣總存款AP', '當年度(ETF)AP', '業績人員', 'yyyymm',
       '基金申購註記', '黃金AUM', '保障型保險AUM', '當年度財務部組合存款AP', 'SNAUM', '當年度投資理財總AP',
       'AUM', 'has_traded', '職業_0', '職業_0.1', '職業_01_金融業', '職業_02_保險業',
       '職業_03_投信／投顧／證券', '職業_04_資訊業', '職業_05_建築業', '職業_06_自由業', '職業_07_製造業',
       '職業_08_貿易買賣業', '職業_09_專業人士', '職業_10_政府機構', '職業_11_各級學校/學術機構',
       '職業_12_軍警機構', '職業_13_家管/退休人員', '職業_14_軍官/軍人', '職業_15_學生/學齡前兒童',
       '職業_16_農林漁牧業', '職業_17_礦石及土石採取業', '職業_18_水電燃氣業', '職業_19_批發及零售業',
       '職業_20_住宿及餐飲業', '職業_21_運輸或倉儲及通訊業', '職業_22_不動產及租賃業', '職業_23_技術服務業',
       '職業_24_賭博/夜總會相關行業', '職業_25_藝品或寶石交

In [11]:
tran_data = pd.read_pickle(DATA_PATH / "trans_buy.pkl")
new_user = crm[crm['has_traded']==0]['身分證字號'].unique().tolist()
len(new_user)

75105

In [12]:
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from sklearn.cluster import Birch, DBSCAN
import itertools
import numpy as np
import umap


n_cluster = 50
n_comp = 10
mx = 0
mx_ncl = 0
mx_nco = 0

def feature_selecter(udata, method, n_components):
    if method not in ["tsne", "umap"]:
        raise ValueError("feature select method not allowed")                    
    
    if method == "tsne":
        x_embedded = TSNE(n_components=n_components, init='random', random_state=47, n_jobs=30).fit_transform(udata.values)
        x_embedded = np.array(x_embedded)
    
    if method == "umap":
        x_embedded = umap.UMAP(n_components=n_components, random_state=47).fit_transform(udata.values)
        
    return x_embedded

def clustering(user_data, x_embedded, method, n_clusters=None, eps=None, min_samples=None):
    """
    quick annotation
    n_clusters for birch
    eps, min_samples for dbscan
    """
    if method not in ["birch", "dbscan"]:
        raise ValueError("clustering method not allowed")
    
    if method == "birch":
        brc = Birch(n_clusters=n_clusters)
        brc.fit(x_embedded)
        brc_x = np.array(brc.predict(x_embedded))
        user_data['cluster'] = brc_x                        
    
    if method == "dbscan":
        dbscan = DBSCAN(
            eps=eps,
            min_samples=min_samples
        ).fit(x_embedded)         
        user_data['cluster'] = dbscan.labels_
    
    return user_data

def recommender_method(tran_data, user_data):
    clusters_top10 = {}
    clusters = user_data.cluster.value_counts().index.tolist()
    for i in clusters:
        userlen = len(user_data[user_data['cluster'] == i])
        tranlen = len(tran_data[tran_data['id_number'].isin( user_data[user_data['cluster'] == i]['身分證字號'])])
        cluster_top10 = tran_data[tran_data['id_number'].isin( 
            user_data[user_data['cluster'] == i]['身分證字號']
        )]['fund_id'].value_counts().index.tolist()[:10]
        clusters_top10[i] = cluster_top10
    
    return clusters_top10
    


In [17]:
crm[crm['身分證字號'].isin(new_user)]
crm.drop(columns=['index'], inplace=True)
crm

,ETFAUM,外幣定存AUM,海外債AUM,IbrainAUM,投資型保險AUM,年收入,台幣活存AUM,年齡,收入註記,總投資理財AUM,...,職業_37_旅遊業,職業_38_社工/義工,職業_39_保全業,職業_41_網路拍賣,職業_42_虛擬貨幣業,職業_99_其他,職業_ND,性別_0.0,性別_1.0,性別_2.0
0,0.0,0.000000,0.0,0.0,0.0,13.081541,13.496670,66,1,12.611538,...,0,0,0,0,0,0,0,0,1,0
1,0.0,0.000000,0.0,0.0,0.0,13.815511,12.279800,66,1,13.787358,...,0,0,0,0,0,0,0,0,1,0
2,0.0,13.218726,0.0,0.0,0.0,14.919774,11.381688,55,1,14.137175,...,0,0,0,0,0,0,0,0,1,0
3,0.0,0.000000,0.0,0.0,0.0,0.000000,13.160068,21,0,13.502971,...,0,0,0,0,0,1,0,0,1,0
4,0.0,0.000000,0.0,0.0,0.0,0.000000,13.239393,21,0,9.210340,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
154948,0.0,0.000000,0.0,0.0,0.0,13.815511,0.000000,48,1,0.000000,...,0,0,0,0,0,0,0,1,0,0
154949,0.0,0.000000,0.0,0.0,0.0,14.220976,0.000000,32,1,0.000000,...,0,0,0,0,0,0,0,1,0,0
154950,0.0,0.000000,0.0,0.0,0.0,12.206073,0.000000,49,1,0.000000,...,0,0,0,0,0,0,0,1,0,0
154951,0.0,0.000000,0.0,0.0,0.0,14.508658,0.000000,43,1,0.000000,...,0,0,0,0,0,0,0,0,0,1


In [18]:
crm.yyyymm.value_counts()

202012    154953
Name: yyyymm, dtype: int64

In [19]:
embedding_method="umap"
n_components=25

user_data = crm
u_data = user_data.drop(columns=['身分證字號', 'CIFAOCODE'])
u_data = u_data.replace(-np.inf, -1)
u_data.fillna(0, inplace=True)        

x_embedded = feature_selecter(u_data, embedding_method, n_components)

/usr/local/lib/python3.6/dist-packages/sklearn/manifold/_spectral_embedding.py:236: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn("Graph is not fully connected, spectral embedding"


In [22]:
c_user_data = clustering(user_data, x_embedded, 'dbscan', eps=6.5, min_samples=5)

In [23]:
cluster_top10 = recommender_method(trans_buy_train, c_user_data)

In [38]:
best_sells

,fund_id,count
0,J1N,4332
1,MB0,4265
2,T34,4056
3,MX5,3451
4,Y38,2886
...,...,...
2095,W25,1
2096,88W,1
2097,80X,1
2098,N61,1


In [46]:
for i in cluster_top10:

    cluster_rec_len = len(cluster_top10[i])
    if cluster_rec_len != 10:
        new_rec = best_sells[~best_sells['fund_id'].isin(cluster_top10[i])].head(10 - cluster_rec_len)['fund_id'].tolist()
        cluster_top10[i] = cluster_top10[i] + new_rec

In [47]:
result_by_cluster = {}
for u in tqdm(new_user):
        temp = user_data[user_data['身分證字號']==u]
        t_cluster = temp.cluster.tolist()[0]        
        result_by_cluster[u] = cluster_top10[t_cluster]


100%|██████████| 75105/75105 [10:59<00:00, 113.91it/s]


In [28]:
list(result_by_cluster.values())[0]

['T34', 'T38', 'Y38', 'Y57', 'L91', 'L0C', 'MX5', 'MB0', 'J1N', 'MU7']

In [29]:
len(result_by_cluster)

75105

In [48]:
import json
r_name = "r_2022_05_01_cluster"
json_name = r_name+".json"
json_save_path = RESULT_PATH / json_name 
with open(json_save_path, 'w') as jf:
    json.dump(result_by_cluster, jf)
from recs.utils.common import convert_result_json_2_csv
csv_name = f"{r_name}.csv"
save_path = RESULT_PATH / csv_name
convert_result_json_2_csv(str(json_save_path), str(save_path))

Convert ../../result/r_2022_05_01_cluster.json to ../../result/r_2022_05_01_cluster.csv success.


In [ ]:
# umap dbscan default 'recall@3': 0.04323324245199246, 'recall@5': 0.05881050636022228, 'recall@10': 0.09080525340057341, 'recall@all': 0.09080525340057341
# umap birch default 'recall@3': 0.05260426783864283, 'recall@5': 0.06474051122488622, 'recall@10': 0.09673095385752385, 'recall@all': 0.09673095385752385
# umap dbscan eps=6.5 min_sample=5 'recall@3': 0.04927221216283717, 'recall@5': 0.06522885652857245, 'recall@10': 0.09996868834525836, 'recall@all': 0.09996868834525836

In [33]:
print(mx, mx_n_components)

0.1024586173976874 25


In [26]:
print(mx, mx_eps)

0.09996868834525836 6.5


In [28]:
# find best min_samples for dbscan
mx = 0
mx_min_samples = 0

for i in range(10):
    r = backtest(user_data, x_embedded, trans_buy_train, cluster_method="dbscan", eps=6.5, min_samples=100*(i+1))
    if r['recall@all'] > mx:
        mx = r['recall@all']
        mx_min_samples = 100*(i+1)

100%|██████████| 2112/2112 [00:00<00:00, 208304.80it/s]


{'recall@3': 0.04927221216283717, 'recall@5': 0.06522885652857245, 'recall@10': 0.09996868834525836, 'recall@all': 0.09996868834525836}


100%|██████████| 2112/2112 [00:00<00:00, 214311.95it/s]


{'recall@3': 0.049745697011322025, 'recall@5': 0.06522885652857245, 'recall@10': 0.09949520349677352, 'recall@all': 0.09949520349677352}


100%|██████████| 2112/2112 [00:00<00:00, 203159.65it/s]


...

100%|██████████| 2112/2112 [00:00<00:00, 206181.22it/s]


{'recall@3': 0.049903525294150305, 'recall@5': 0.06522885652857245, 'recall@10': 0.099542551981622, 'recall@all': 0.099542551981622}


100%|██████████| 2112/2112 [00:00<00:00, 214873.38it/s]


{'recall@3': 0.049903525294150305, 'recall@5': 0.06428188683160274, 'recall@10': 0.09881315984597985, 'recall@all': 0.09881315984597985}


KeyboardInterrupt: 

In [18]:
backtest(user_data, x_embedded, trans_buy_train, cluster_method="dbscan", eps=1, min_samples=5)

100%|██████████| 2112/2112 [00:00<00:00, 197572.71it/s]

{'recall@3': 0.04283867174492175, 'recall@5': 0.059520733632949546, 'recall@10': 0.09025285441067442, 'recall@all': 0.09025285441067442}


In [17]:
backtest(user_data, x_embedded, trans_buy_train, cluster_method="dbscan", eps=0.5, min_samples=5)


100%|██████████| 2112/2112 [00:00<00:00, 194139.04it/s]

{'recall@3': 0.042759757603507614, 'recall@5': 0.057968755518471436, 'recall@10': 0.08915932987964989, 'recall@all': 0.08915932987964989}


In [12]:
backtest(user_data, x_embedded, trans_buy_train, cluster_method="birch")

100%|██████████| 2112/2112 [00:00<00:00, 202033.71it/s]

{'recall@3': 0.05260426783864283, 'recall@5': 0.06474051122488622, 'recall@10': 0.09673095385752385, 'recall@all': 0.09673095385752385}


In [25]:
import hdbscan

ModuleNotFoundError: No module named 'sklearn.externals.joblib'

In [13]:
embedding_method="tsne"
n_components=3

user_data = crm[crm.yyyymm == 201912]
u_data = user_data.drop(columns=['身分證字號', 'CIFAOCODE'])
u_data = u_data.replace(-np.inf, -1)
u_data.fillna(0, inplace=True)

x_embedded = feature_selecter(u_data, embedding_method, n_components)
backtest(user_data, x_embedded, trans_buy_train, cluster_method="birch")
backtest(user_data, x_embedded, trans_buy_train, cluster_method="dbscan")

100%|██████████| 2112/2112 [00:00<00:00, 200619.86it/s]


{'recall@3': 0.050019670722795714, 'recall@5': 0.06359495801967394, 'recall@10': 0.09453037665694668, 'recall@all': 0.09453037665694668}


100%|██████████| 2112/2112 [00:00<00:00, 187558.12it/s]

{'recall@3': 0.033577186702186713, 'recall@5': 0.04223782583157584, 'recall@10': 0.07180861057580556, 'recall@all': 0.07180861057580556}


In [49]:
len(list(set(target_users) & set(new_user)))

2112

In [51]:
len(new_real)

11056

In [37]:
print(cluster_top10)

[995, 921, 2940, 20, 722, 202, 2732, 2128, 3197, 985]


In [48]:
# umap birch 0.01971914093398498 50 12
print(mx, mx_ncl, mx_nco)

0.01971914093398498 50 12
